<a id='top'></a>
# Full Calibration Pipeline
---

Run the stage 1 pipeline on all the relevant reference science, PSF and background files

- Science observations: 8, 9
- Science backgrounds: 30
- Reference observations: 7
- Reference backgrounds: 31

**Author**: Jonathan Aguilar (jaguilar@stsci.edu) | **Latest Update**: 25 Oct 2023

* [Introduction](#intro)
* [Pipeline Resources and Documentation](#resources)
* [Imports](#imports)
* [Convenience tools](#convenience_tools)
* [File selection](#file_selection)
* [Run the pipeline stages](#pipeline)
    * [Detector1](#detector1)
    * [Image2](#image2)
    * [Coron3](#coron3)

<a id='intro'></a>
## Introduction

This notebook runs a complete set of observations through the all the pipeline stages. This includes:
- 2 observations of the science target at different roll angles (1 exposure each)
- 1 observation of the PSF reference target, taken with a 9-pt dither pattern (9 exposures total)
- 2 background observations associated with each star, each taken with a 2-pt dither pattern (2 groups of 2 observations).


We will construct our own association files that the pipeline will use to perform background and PSF subtraction. 

## Required folder structure

This notebook depends on the following folder structure for input and output:
- `./uncal/` -> location for the downloaded uncal files
- `./full_pipeline_output/` -> for association files and stage output folders
    - `stage1/` -> for `calwebb_detector1` output
    - `stage2/` -> for `calwebb_image2` output
    - `stage3/` -> for `calwebb_coron3` output    

<a id='resources'></a>
## Pipeline resources and documentation

Documentation on the stages and steps run on MIRI coronagraphy data specifically can be found here: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/main.html#pipelines
- `calwebb_detector1` https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_detector1.html#calwebb-detector1
- `calwebb_image2` https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_image2.html#calwebb-image2
- `calwebb_coron3` https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_coron3.html#calwebb-coron3

For science users, the most important stages are `calwebb_detector1` and `calwebb_image2`. These convert the data from raw, uncalibrated detector numbers into datacubes with units of MJy/sr. `calwebb_coron3` is primarily a tool for preliminary examination of the data.

<a id='imports'></a>
## Imports

In [ ]:
import os
from collections import OrderedDict
from pathlib import Path

In [ ]:
import numpy as np

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
from astropy.io import fits

<a id='convenience_tools'></a>
## Convenience tools

Environment paths and functions that make life easier.

First, set up a local CRDS directory. When the pipeline pulls a reference file from CRDS for the first time, it will write a copy to this directory. All subsequent reads of the reference file will redirect to the local directory instead of sending the file again over the network.

Use the local repository of reference files because it's faster

See https://jwst-pipeline.readthedocs.io/en/latest/jwst/user_documentation/reference_files_crds.html#crds

In [ ]:
os.environ['CRDS_PATH'] = '/Volumes/agdisk/crds/'
# os.environ['CRDS_PATH'] = ''
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

Advanced users - uncomment the cell below and specify the context if you have a specific combination of reference files you want to use

In [ ]:
# os.environ['CRDS_CONTEXT'] = 'jwst_1140.pmap'

In [ ]:
# some plot formatting
mpl.rcParams['image.origin'] = "lower"

In [ ]:
# you may need to run this command twice for plots to pop up correctly
# %matplotlib auto
%matplotlib inline

In [ ]:
# this helper function to map out a particular DQ flag will be useful
# DQ flags can be found here: 
# https://jwst.readthedocs.io/en/latest/jwst/references_general/references_general.html#data-quality-flags
def get_dq_flag(flag, dq_img):
    """return the pixels that have a given DQ flag"""
    bad_bitvalue = dqflags.pixel[flag]
    flags = np.bitwise_and(dq_img, bad_bitvalue).astype(bool)
    return flags

<a id="file_selection"></a>
## Select a file for processing

Please replace the path in the cell below with a path to a file on your own system. 

If you would like to use this specific exposure as an example, you can retrieve it from MAST as follows:
```
filenames = [
    'jw01386004001_04101_00001_mirimage_uncal.fits',  # Science target, roll 1
    'jw01386005001_04101_00001_mirimage_uncal.fits',  # Science target, roll 2
    'jw01386006001_04101_00001_mirimage_uncal.fits',  # PSF reference star observations
    'jw01386006001_04101_00002_mirimage_uncal.fits',
    'jw01386006001_04101_00003_mirimage_uncal.fits',
    'jw01386006001_04101_00004_mirimage_uncal.fits',
    'jw01386006001_04101_00005_mirimage_uncal.fits',
    'jw01386006001_04101_00006_mirimage_uncal.fits',
    'jw01386006001_04101_00007_mirimage_uncal.fits',
    'jw01386006001_04101_00008_mirimage_uncal.fits',
    'jw01386006001_04101_00009_mirimage_uncal.fits',
    'jw01386028001_02101_00001_mirimage_uncal.fits',  # Science backgrounds
    'jw01386028001_03101_00001_mirimage_uncal.fits',
    'jw01386029001_02101_00001_mirimage_uncal.fits',  # PSF reference star backgrounds
    'jw01386029001_03101_00001_mirimage_uncal.fits',
]
from astroquery.mast import Observations
for filename in filenames:
    Observations.download_file(f"mast:JWST/product/{filename}", local_path= f"./uncal/{filename}")
uncal_file = f"./uncal/{filenames[0]}"
```

In [ ]:
filenames = [
    'jw01386008001_04101_00001_mirimage_uncal.fits',  # Science target, roll 1
    'jw01386009001_04101_00001_mirimage_uncal.fits',  # Science target, roll 2
    'jw01386007001_04101_00001_mirimage_uncal.fits',  # PSF reference star observations
    'jw01386007001_04101_00002_mirimage_uncal.fits',
    'jw01386007001_04101_00003_mirimage_uncal.fits',
    'jw01386007001_04101_00004_mirimage_uncal.fits',
    'jw01386007001_04101_00005_mirimage_uncal.fits',
    'jw01386007001_04101_00006_mirimage_uncal.fits',
    'jw01386007001_04101_00007_mirimage_uncal.fits',
    'jw01386007001_04101_00008_mirimage_uncal.fits',
    'jw01386007001_04101_00009_mirimage_uncal.fits',
    'jw01386030001_02101_00001_mirimage_uncal.fits',  # Science backgrounds
    'jw01386030001_03101_00001_mirimage_uncal.fits',
    'jw01386031001_02101_00001_mirimage_uncal.fits',  # PSF reference star backgrounds
    'jw01386031001_03101_00001_mirimage_uncal.fits',
]
from astroquery.mast import Observations
for filename in filenames:
    Observations.download_file(f"mast:JWST/product/{filename}", local_path= f"./uncal/{filename}")

<a id='pipieline'></a>
## Calibration pipeline

In [ ]:
import jwst
jwst.__version__

In [ ]:
from jwst.pipeline import (
    Detector1Pipeline, 
    Image2Pipeline,
    Coron3Pipeline
)

In [ ]:
output_dir = Path("full_pipeline_output")

<a id='detector1'></a>
### Stage 1: Detector corrections

We're going to write out the results of each step to disk, and also keep a copy in memory in the `results` dict generated in the cell below. Make sure you increase the maximum number of cores to at least quarter or half.

In [ ]:
for i, f in enumerate(filenames):
    # run det1
    print("\n\n\n", i+1, len(filenames), "\n\n\n")
    filename = str(Path("uncal") / f)
    res1 = Detector1Pipeline.call(
        filename,
        output_dir=str(output_dir / "stage1"),
        save_results=True,
        steps={
            'jump': {'maximum_cores': 'half'},
            'ramp_fit': {'maximum_cores': 'half'}
        }
    )

<a id='image2'></a>
### Stage 2: Photometric calibration

In [ ]:
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase

In [ ]:
det1_files = sorted((output_dir / "stage1").glob("*rateints.fits"))
# remove TACQ exposures
for i in [i for i, f in enumerate(det1_files) if fits.getval(f, "EXP_TYPE", 0) == "MIR_TACQ"][::-1]:
    det1_files.pop(i)

In [ ]:
# separate the files into star and background exposures using the BKGDTARG keyword
files = {'sci': [], 'sci-bgnd': [], 
         'psf': [], 'psf-bgnd': []}
for f in det1_files:
    obsnum = int(fits.getval(f, 'OBSERVTN', 0))
    if obsnum in [8, 9]:
        files['sci'].append(str(f.resolve()))
    elif obsnum == 7:
        files['psf'].append(str(f.resolve()))
    elif obsnum == 30:
        files['sci-bgnd'].append(str(f.resolve()))
    elif obsnum == 31:
        files['psf-bgnd'].append(str(f.resolve()))
    else:
        pass

In [ ]:
# to generate a proper background-subtracting association file, 
# first make one with just the star exposures, and then add the background 
# files by hand
asn_sci = asn_from_list.asn_from_list(files['sci'], rule=DMSLevel2bBase)
asn_psf = asn_from_list.asn_from_list(files['psf'], rule=DMSLevel2bBase)

In [ ]:
asn_sci

This is what the ASN file looks like, with only the PSF star observations included

In [ ]:
# for each dataproduct in the products, add the background files by hand
for product in asn_sci['products']:
    for f in files['sci-bgnd']:
        product['members'].append({'expname': f, 'exptype': 'background'})
for product in asn_psf['products']:
    for f in files['psf-bgnd']:
        product['members'].append({'expname': f, 'exptype': 'background'})        

In [ ]:
# write this out to a json file
# It's going to complain about paths, but ignore it
with open(output_dir / 'sci-coron_bkgsubtest_asn.json', 'w') as fp:
    fp.write(asn_sci.dump()[1])
with open(output_dir / 'psf-coron_bkgsubtest_asn.json', 'w') as fp:
    fp.write(asn_psf.dump()[1])    

In [ ]:
# you can get a list of available options with `Image2Pipeline().get_pars()`
params = {
    'output_dir': str(output_dir / "stage2"),
    'save_results': True,
    'save_bsub': True,
    'steps': {
        'bkg_subtract': {
            'save_combined_background': True
        },
    }
}

img2 = Image2Pipeline.call(
    str(output_dir / "sci-coron_bkgsubtest_asn.json"),
    **params,
)
img2 = Image2Pipeline.call(
    str(output_dir / "psf-coron_bkgsubtest_asn.json"),
    **params,
)

<a id='coron3'></a>
### Stage 3: PSF subtraction

To run calwebb_coron3, we have to generate an association file and tell it which exposures belong to the science target and which belong to the PSF reference star.

In [ ]:
img2_files = sorted((output_dir / "stage2").glob("*calints.fits"))

In [ ]:
# Sort the files into science or PSF target, using the IS_PSF header keyword
files = {
    'sci': [],
    'psf': []
}

for f in img2_files:
    is_psf = fits.getval(f, 'IS_PSF', 0)
    if is_psf == True:
        files['psf'].append(str(f.resolve()))
    else: # is_psf == False
        files['sci'].append(str(f.resolve()))
        
files

The steps, in order of execution, are:


We're going to write out the results of each step to disk, and also keep a copy in memory in the `results` dict generated in the cell below.

In [ ]:
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base

In [ ]:
# to generate a proper background-subtracting association file, 
# first make one with just the science files and then add the background files
# by hand
asn = asn_from_list.asn_from_list(files['sci'], rule=DMS_Level3_Base, product_name='test')
for f in files['psf']:
    asn['products'][0]['members'].append({'expname': str(f), 'exptype': 'psf'})

#### 

In [ ]:
# write this out to a json file
with open(str(output_dir / 'coron3_asn.json'), 'w') as fp:
    fp.write(asn.dump()[1])

Run Coron3 with default parameters.

In [ ]:
# you can get a list of available options with `Coron3Pipeline().get_pars()`
params = {
    'output_dir': str(output_dir / "stage3"),
    'save_results': True
}

cor3 = Coron3Pipeline.call(
    str(output_dir / 'coron3_asn.json'),
    **params,
)